# Entrenamiento y evaluación de modelos
## Contratación de Seguros de Viajes  
  
  
  
#### Preprocesamiento de los datos  
Como primera instancia, aplicamos las transformaciones sobre el dataset que indicamos en el trabajo práctico anterior.

In [ ]:
# Importamos las dependencias necesarias.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn_pandas
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from collections import defaultdict
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# Importamos el dataset.
dataset = pd.read_csv('TravelInsurancePrediction.csv')

# Renombramos las variables.
dataset = dataset.rename(columns={'Idx':'idx', 'Age':'age', 'Employment Type':'employment_type', 'GraduateOrNot':'graduate_or_not', 'AnnualIncome':'annual_income', 'FamilyMembers':'family_members', 'ChronicDiseases':'chronic_diseases', 'EverTravelledAbroad':'ever_travelled_abroad', 'FrequentFlyer':'frequent_flyer', 'TravelInsurance':'travel_insurance'})

# Seleccionamos la primer columna 'idx', como índice.
dataset = dataset.set_index("idx")

# Reemplazamos los valores 'Yes' y 'No' por 0 y 1 de las variables indicadas en el tp1.
dataset["graduate_or_not"] = dataset.graduate_or_not.replace(['No', 'Yes'], [0,1])
dataset["frequent_flyer"] = dataset.frequent_flyer.replace(['No', 'Yes'], [0,1])
dataset["ever_travelled_abroad"] = dataset.ever_travelled_abroad.replace(['No', 'Yes'], [0,1])

# Reemplazamos los dos posibles valores de la columna 'employment_type' por 0 y 1.
dataset["employment_type"] = dataset.employment_type.replace(['Private Sector/Self Employed', 'Government Sector'], [0,1])

In [ ]:
# Dividimos el dataset en train (60%), test (20%) y validation (20%)
train, not_train = train_test_split(dataset, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

In [ ]:
# Escalamos las variables numéricas.
mapper_scaller = DataFrameMapper([
    (['age'],[MinMaxScaler()]),
    (['employment_type'],None),
    (['graduate_or_not'],None),
    (['annual_income'],[MinMaxScaler()]),
    (['family_members'],[MinMaxScaler()]),
    (['chronic_diseases'],None),
    (['frequent_flyer'],None),
    (['ever_travelled_abroad'],None)
])

mapper_scaller.fit(train)

#### Elección de una métrica  
La métrica que utilizaremos es Accuracy, debido a que permite medir el porcentaje de casos acertados en la predicción, siendo un concepto simple de explicar al cliente que requiere el modelo. Además es útil especialmente en problemas de clasificación, como es este caso.  
Por otra parte, a modo de uso interno, también podríamos tener en cuenta la métrica llamada Precission, ya que a pesar de que la misma no sería comunicada al cliente directamente para evitar confundirlo con conceptos un tanto abstractos, nos permitiría tener en cuenta los casos de 'falsos positivos'. Estos casos serían personas que el modelo predice como potenciales compradoras del seguro pero que en la realidad no lo son, y es importante conocerlos debido a que implicarían una posible pérdida de dinero para la empresa al no realizar mayores intentos de convencerlos de lo contrario, por pensar que no lo necesitan.


#### Elección de una técnica de feature engineering  
Teniendo en cuenta las técnicas vistas en clase, decidimos optar por crear una nueva feature extrayendo información de la columna 'annual_income', con el objetivo de simplificar los datos para el modelo. La idea sería categorizar a los ejemplos entre clientes de clases alta, media y baja, en función de sus ingresos. Según diversas fuentes consultadas, consideramos a la clase media como aquellos que poseen ingresos anuales entre 400.000 rupias y 1.200.000 rupias. Por encima de ese rango serían personas de clase alta, y por debajo, clase baja.  
Luego de ésto al tener una columna con variables categóricas, aplicaríamos la técnica 'One-Hot Encoder' para convertirla en 3 columnas binarias que tomen valores '0' o '1', indicando si es de esa clase o no.

In [ ]:
# Función que devuelve la clase social en función de los ingresos.
def get_social_class(row):
    if(row['annual_income'] > 1200000):
        return 'alta'
    elif(row['annual_income'] < 400000):
        return 'baja'
    else:
        return 'media'
    
# Agregamos la nueva columna al dataset.
social_class = dataset.apply(get_social_class,axis=1)
dataset_with_fe = dataset
dataset_with_fe['social_class'] = social_class

dataset_with_fe

In [ ]:
# Dividimos el dataset en train (60%), test (20%) y validation (20%)
train_fe, not_train_fe = train_test_split(dataset_with_fe, test_size=0.4, random_state=42)
validation_fe, test_fe = train_test_split(not_train_fe, test_size=0.5, random_state=42)

In [ ]:
# Escalamos las variables numéricas.
mapper_fe = DataFrameMapper([
    (['age'],[MinMaxScaler()]),
    (['employment_type'],None),
    (['graduate_or_not'],None),
    (['annual_income'],[MinMaxScaler()]),
    (['family_members'],[MinMaxScaler()]),
    (['chronic_diseases'],None),
    (['frequent_flyer'],None),
    (['ever_travelled_abroad'],None),
    (['social_class'],[OneHotEncoder()])
])

mapper_fe.fit(train_fe)

#### Entrenamiento y evaluación de modelos  
Para este dataset seleccionamos los siguientes modelos:
* k-Nearest Neighbors o k-NNN (vecinos cercanos)
* Decision Tree (árbol de decisión)
* Gradient Boosting

In [ ]:
# Función para evaluar modelos (sin feature engineering).
def evaluate_model(model, set_names=('train', 'validation'), title=''):
    if title:
        display(title)
        
    final_metrics = defaultdict(list)
    
    for i, set_name in enumerate(set_names):
        assert set_name in ['train', 'validation', 'test']
        set_data = globals()[set_name] 

        y = set_data.travel_insurance
        y_pred = model.predict(set_data)
        final_metrics['Accuracy'].append(metrics.accuracy_score(y, y_pred))
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred))
        
    display(pd.DataFrame(final_metrics, index=set_names))

In [ ]:
# Función para evaluar modelos (con feature engineering).
def evaluate_model_fe(model, set_names=('train_fe', 'validation_fe'), title=''):
    if title:
        display(title)
        
    final_metrics = defaultdict(list)
   
    for i, set_name in enumerate(set_names):
        assert set_name in ['train_fe', 'validation_fe', 'test_fe']
        set_data = globals()[set_name] 

        y = set_data.travel_insurance
        y_pred = model.predict(set_data)
        final_metrics['Accuracy'].append(metrics.accuracy_score(y, y_pred))
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred))
        
    display(pd.DataFrame(final_metrics, index=set_names))

Antes de comenzar con el análisis y evaluación de los modelos, debemos aclarar que los valores de métricas obtenidos, en la mayoría de los casos, difieren en valores decimales, por lo cual los distintos cambios que fuimos aplicando no generan un gran impacto en dichos estimadores, al tratarse de valores pequeños. Sin embargo, decidimos analizar estas pequeñas diferencias para así poder detectar relaciones.

##### *Sin feature engeeniring ni alteración de hiperparámetros.*  
En primera instancia, evaluamos los 3 modelos elegidos sin haber implementado la feature engineering seleccionada y sin modificar los hiper-parámetros que vienen por defecto. 
A partir de los valores obtenidos para “Accuracy” en el set de datos de validation, podemos observar que el mejor modelo en este caso sería Gradient Boosting, ya que presenta el mayor valor de ésta métrica en el set de datos de validation con respecto a los demás modelos y con una diferencia casi nula con el resultado obtenido en el set de datos de train. Además, es posible que los demás modelos estén sobreentrenando, especialmente el árbol, ya que la diferencia de la métrica obtenida para el train y para el validation es bastante notoria.

In [ ]:
# kNN
modelKNN = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', KNeighborsClassifier())
])
modelKNN.fit(train, train.travel_insurance)
evaluate_model(modelKNN, title='kNN')

# Decision Tree
modelTree = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', DecisionTreeClassifier())
])
modelTree.fit(train, train.travel_insurance)
evaluate_model(modelTree, title='Decision Tree')

# Gradient Boosting
modelGradBoost = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(random_state=42))
])
modelGradBoost.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost, title='Gradient Boosting')

##### *Con feature engeeniring, sin alteración de hiperparámetros.*
En segunda instancia, volvimos a evaluar los 3 modelos aún sin modificar los parámetros por defecto, pero esta vez implementando feature engineering, tal como explicamos anteriormente. En este caso, pudimos observar que los valores son prácticamente los mismos que en la instancia anterior, incluso llegando a empeorar el valor del Accuracy. Nuevamente llegamos a la conclusión de que el mejor modelo entre los seleccionados sería el Gradient Boosting. En general, esta técnica de feature engineering, por sí sola, no aporta mejoras a los modelos. Además podemos observar que el árbol continúa sobreentrenando.

In [ ]:
# kNN + feature engineering
modelKNN_fe = Pipeline([
    ('mapper', mapper_fe),
    ('classifier', KNeighborsClassifier())
])
modelKNN_fe.fit(train_fe, train_fe.travel_insurance)
predictions = modelKNN_fe.predict(validation_fe)
evaluate_model_fe(modelKNN_fe, title='kNN with feature engineering')

# Decision Tree + feature engineering
modelTree_fe = Pipeline([
    ('mapper', mapper_fe),
    ('classifier', DecisionTreeClassifier())
])
modelTree_fe.fit(train_fe, train_fe.travel_insurance)
evaluate_model_fe(modelTree_fe, title='Decision Tree with feature engineering')

# Gradient Boosting + feature engineering
modelGradBoost_fe = Pipeline([
    ('mapper', mapper_fe),
    ('classifier', GradientBoostingClassifier(random_state=42))
])
modelGradBoost_fe.fit(train_fe, train_fe.travel_insurance)
evaluate_model_fe(modelGradBoost_fe, title='Gradient Boosting with feature engineering')

##### *Sin feature engeeniring, alterando hiperparámetros.*
En tercera instancia, evaluamos los 3 modelos elegidos sin haber implementado feature engineering, pero modificando los hiper-parámetros que vienen por defecto.   
  
Con respecto al modelo k-NN, observamos que por defecto venía con un k=5, por lo que decidimos probar con k=3, k=4, k=6, k=7 y k=10. Una vez evaluado cada uno de ellos, pudimos observar que al utilizar un k=4, mejora mínimamente el valor de la métrica en comparación con el k por defecto. Por otra parte, pudimos notar la dependencia e inestabilidad de este modelo en k ya que pequeñas modificaciones en esta, generan valores diversos de la métrica. Con respecto a las k probadas, teniendo en cuenta el valor por defecto de k (5), con k en 4 la métrica mejora, pero también lo hace con 6 y con 10, y por otro lado con 3 y 7 empeora.

In [ ]:
# KNN k=3
modelKNN3 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', KNeighborsClassifier(3))
])
modelKNN3.fit(train, train.travel_insurance)
evaluate_model(modelKNN3, title='kNN with k=3')

# KNN k=4
modelKNN4 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', KNeighborsClassifier(4))
])
modelKNN4.fit(train, train.travel_insurance)
evaluate_model(modelKNN4, title='kNN with k=4')

# KNN k=6
modelKNN6 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', KNeighborsClassifier(6))
])
modelKNN6.fit(train, train.travel_insurance)
evaluate_model(modelKNN6, title='kNN with k=6')

# KNN k=7
modelKNN7 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', KNeighborsClassifier(7))
])
modelKNN7.fit(train, train.travel_insurance)
evaluate_model(modelKNN7, title='kNN with k=7')

# KNN k=10
modelKNN10 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', KNeighborsClassifier(10))
])
modelKNN10.fit(train, train.travel_insurance)
evaluate_model(modelKNN10, title='kNN with k=10')

Continuando con el modelo de árbol de decisión, que viene por defecto sin límite de profundidad, le asignamos a éste diferentes valores tales como 4, 8 y 12. Haciendo esto, pudimos comprobar que, tal como dice la teoría, al disminuir la profundidad máxima del árbol se puede llegar a mejores resultados, y se reduce la posibilidad de que dicho modelo sobreentrene. Encontramos como más performante al árbol con una profundidad igual a 4, ya que con ella se obtiene un mejor valor de métrica que con las demás profundidades.

In [ ]:
# tree depth=4
modelTree4 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', DecisionTreeClassifier(max_depth = 4))
])
modelTree4.fit(train, train.travel_insurance)
evaluate_model(modelTree4, title='Decision Tree with depth=4')

# tree depth=8
modelTree8 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', DecisionTreeClassifier(max_depth = 8))
])
modelTree8.fit(train, train.travel_insurance)
evaluate_model(modelTree8, title='Decision Tree with depth=8')

# tree depth=12
modelTree12 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', DecisionTreeClassifier(max_depth = 12))
])
modelTree12.fit(train, train.travel_insurance)
evaluate_model(modelTree12, title='Decision Tree with depth=12')

Por último, evaluamos al modelo Gradient Boosting, alterando la cantidad de árboles, la profundidad máxima y el learning rate, primero por separado y después combinándolos.  
  
En primer lugar probamos el modelo con distintas cantidades de árboles. Sabiendo que el valor por defecto es 100, trabajamos con 10, 50, 100, 150 y 250. A partir de los valores obtenidos podemos concluir que los valores que mejoran en mayor medida la métrica son 10 y 50, mientras que valores mayores a 100 la empeoran.

In [ ]:
# gradient boosting n_trees=10
modelGradBoost10 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(n_estimators=10, random_state=42))
])
modelGradBoost10.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost10, title='Gradient Boosting with n_trees=10')

# gradient boosting n_trees=50
modelGradBoost50 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(n_estimators=50, random_state=42))
])
modelGradBoost50.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost50, title='Gradient Boosting with n_trees=50')

# gradient boosting n_trees=150
modelGradBoost150 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(n_estimators=150, random_state=42))
])
modelGradBoost150.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost150, title='Gradient Boosting with n_trees=150')

# gradient boosting n_trees=250
modelGradBoost250 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(n_estimators=250, random_state=42))
])
modelGradBoost250.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost250, title='Gradient Boosting with n_trees=250')

En segundo lugar probamos el estimador con distintas profundidades para los árboles. Sabiendo que la profundidad por defecto es 3, trabajamos con 4, 6 y 8. A partir de los valores obtenidos, concluimos que la profundidad 3 y 4 son las que mejor funcionan con el modelo. Mientras que, si esta profundidad aumenta, el valor de la métrica empeora, como se puede ver con 6 y 8.

In [ ]:
# gradient boosting depth=4
modelGradBoost4 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(max_depth=4, random_state=42))
])
modelGradBoost4.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost4, title='Gradient Boosting with depth=4')

# gradient boosting depth=6
modelGradBoost6 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(max_depth=6, random_state=42))
])
modelGradBoost6.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost6, title='Gradient Boosting with depth=6')

# gradient boosting depth=8
modelGradBoost8 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(max_depth=8, random_state=42))
])
modelGradBoost8.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost8, title='Gradient Boosting with depth=8')

En tercer lugar probamos el modelo con distintos valores de learning rate. Sabiendo que el valor por defecto es 0,1, trabajamos con 0,01, 0,05, 0,5, 1 y 2. A partir de ello podemos concluir que los valores menores al valor por defecto mejoran el valor del Accuracy, mientras que los valores mayores al valor por defecto lo empeoran. Incluso, un learning rate de 2 hace que el modelo no pueda converger, ya que, según la teoría, un valor de este muy grande genera una no convergencia y va empeorando con el tiempo.

In [ ]:
# gradient boosting lr=0,01
modelGradBoost0_01 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(learning_rate=0.01, random_state=42))
])
modelGradBoost0_01.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost0_01, title='Gradient Boosting with learning_rate=0,01')

# gradient boosting lr=0,05
modelGradBoost0_05 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(learning_rate=0.05, random_state=42))
])
modelGradBoost0_05.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost0_05, title='Gradient Boosting with learning_rate=0,05')

# gradient boosting lr=0,5
modelGradBoost0_5 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(learning_rate=0.5, random_state=42))
])
modelGradBoost0_5.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost0_5, title='Gradient Boosting with learning_rate=0,5')

# gradient boosting lr=1
modelGradBoost1 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(learning_rate=1, random_state=42))
])
modelGradBoost1.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost1, title='Gradient Boosting with learning_rate=1')

# gradient boosting lr=2
modelGradBoost2 = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(learning_rate=2, random_state=42))
])
modelGradBoost2.fit(train, train.travel_insurance)
evaluate_model(modelGradBoost2, title='Gradient Boosting with learning_rate=2')

Finalmente, a partir de los valores anteriores, decidimos combinar estos cambios, probando con aquellos que mejoraban en mayor medida los valores del Accuracy. Entonces, trabajamos con 50 como cantidad de árboles, 3 como profundidad de cada árbol y 0,01 como learning rate. Llegamos a la conclusión de que, estos cambios en conjunto, no mejoran más de lo que lo hacen la cantidad de árboles y el learning rate por separado.

In [ ]:
# gradient boosting n_tree=50, depth=3, lr=0,01
modelGradBoostMix = Pipeline([
    ('mapper', mapper_scaller),
    ('classifier', GradientBoostingClassifier(n_estimators=50, max_depth=3, learning_rate=0.01, random_state=42))
])
modelGradBoostMix.fit(train, train.travel_insurance)
evaluate_model(modelGradBoostMix, title='Gradient Boosting with n_tree=50, depth=3 and lr=0,01')

Por otra parte, para estos casos, el estimador Gradient Boosting sigue siendo la mejor opción a partir del valor del Accuracy obtenido.

##### *Con feature engeeniring, alterando hiperparámetros.*
Como última instancia, decidimos combinar la aplicación de feature engineering con la alteración de los hiper-parámetros, de los cuales seleccionamos aquellos que mejor valor de métrica generaban. Entonces trabajamos con un kNN con k=4, con un árbol con profundidad 4 y un Gradient Boosting con 50 árboles y learning rate de 0,01; todos ellos utilizando la técnica de feature engineering seleccionada.  
  
Con respecto al modelo kNN, al aplicar ambas técnicas pudimos notar una leve mejoría en Accuracy.
En cambio, para el árbol de decisión se obtuvieron los mismos valores de la métrica que cuando solo se habían aplicado modificaciones en los hiper-parámetros. Esta misma situación ocurre con Gradient Boosting, ya que al aplicar las dos técnicas juntas no se aumenta el valor de dicha métrica.  
  
En general, podemos decir, que la modificación de los hiper-parámetros por sí sola, es suficiente para mejorar la performance de los modelos. Esto puede deberse a que la técnica de feature engineering seleccionado no fue óptima.

In [ ]:
# KNN fe, k=4
modelKNN_fe_hp = Pipeline([
    ('mapper', mapper_fe),
    ('classifier', KNeighborsClassifier(4))
])
modelKNN_fe_hp.fit(train_fe, train_fe.travel_insurance)
predictions = modelKNN_fe.predict(validation_fe)
evaluate_model_fe(modelKNN_fe_hp, title='kNN with feature engineering and k=4')

# decision tree fe, depth=4
modelTree_fe_hp = Pipeline([
    ('mapper', mapper_fe),
    ('classifier', DecisionTreeClassifier(max_depth=4))
])
modelTree_fe_hp.fit(train_fe, train_fe.travel_insurance)
evaluate_model_fe(modelTree_fe_hp, title='Decision Tree with feature engineering and depth=4')

# gradient boosting fe, n_trees=50
modelGradBoost_fe_50 = Pipeline([
    ('mapper', mapper_fe),
    ('classifier', GradientBoostingClassifier(n_estimators=50, random_state=42))
])
modelGradBoost_fe_50.fit(train_fe, train_fe.travel_insurance)
evaluate_model_fe(modelGradBoost_fe_50, title='Gradient Boosting with feature engineering and n_trees=50')

# gradient boosting fe, lr=0,01
modelGradBoost_fe_0_01 = Pipeline([
    ('mapper', mapper_fe),
    ('classifier', GradientBoostingClassifier(n_estimators=50, random_state=42))
])
modelGradBoost_fe_0_01.fit(train_fe, train_fe.travel_insurance)
evaluate_model_fe(modelGradBoost_fe_0_01, title='Gradient Boosting with feature engineering and lr=0,01')

Una vez más, el estimador Gradient Boosting sigue siendo la mejor opción a partir del valor del Accuracy obtenido.

#### Algunas técnicas para no sobreentrenar
* En primer lugar dividimos el dataset en train, test y validation para que los modelos entrenen con un conjunto diferente al que utilizarán para predecir.
* También aplicamos modificaciones sobre los hiperparámetros en los diversos modelos, como por ejemplo, establecer una profundidad en el árbol o modificar el learning rate del Gradient Boosting.
* Otra opción podría ser la de agregar más ejemplos al dataset, lo cual no pudimos aplicar ya que no disponemos de los mismos. Un ejemplo podría ser el del árbol de decisión sin ninguna modificación. Como podemos ver en el gráfico siguiente, la línea del train y del validation no convergen, por lo que podríamos decir que está sobreentrenando. Sin embargo, a medida que aumenta el tamaño del dataset, estas líneas se van acercando, lo que indica que si agregaríamos más datos, las mismas podrían llegar a converger en algún punto, reduciendo el sobreentrenamiento.

In [ ]:
# Gráfico Learning Curve
train_sizes, train_scores, test_scores = learning_curve(
estimator=modelTree,
X=train,
y=train.travel_insurance,
train_sizes=np.linspace(0.1, 1.0, 10),
cv=10,
n_jobs=-1,
scoring='accuracy')

train_mean = np.mean(train_scores, axis=1)

test_mean = np.mean(test_scores, axis=1)

plt.plot(train_sizes, train_mean, color='#0000FF', label='Training')
plt.plot(train_sizes, test_mean, color='#FF0000', label='Validation')

plt.title('Learning Curve - Decision Tree')
plt.xlabel('Tamaño del train')
plt.ylabel('Accuracy')
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

#### Selección de un modelo y valor de la métrica a informar  
El modelo seleccionado fue el Gradient Boosting con [un learning rate de 0,01/una cantidad de estimadores igual a 50], ya que es el que mayor valor de Accuracy tuvo en el dataset de validation.  
Cabe aclarar que también fuimos controlando que el valor de Precision se mantenga en valores altos, si bien hubo un modelo que obtuvo una mejor Precision (por poco), lo que marcó la decisión fue la primera métrica mencionada, ya que es la que informaremos al cliente. De cualquier forma, obtuvimos buenos valores para ambas métricas, siendo aproximadamente un 0,8463 en Accuracy y un 0,9474 en Precision, para el dataset de validation. Esto quiere decir que el estimador acertó en un 84,63% de las predicciones y en un 94,74% de los ejemplos que predijo como verdaderos (es decir, de los clientes que predijo que iban a comprar el seguro, los que realmente lo hicieron).  
Para el caso del Precision, nos interesaría enfocarnos en ese 5,26%, que son aquellos casos que según el estimador iban a comprar, pero no lo hicieron. 
Sin embargo, como mencionamos antes, la métrica más óptima para informarle al cliente es Accuracy, y este valor se obtiene del dataset de test. Entonces, **el valor a informar es del 84,63%** aproximadamente.

In [ ]:
evaluate_model(modelGradBoost0_01, title='Gradient Boosting with lr=0,01', set_names=('train', 'validation','test'))
evaluate_model(modelGradBoost50, title='Gradient Boosting with n_trees=50', set_names=('train', 'validation','test'))